<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/dbd_notebook_ipynb_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DBD Plasma Actuator Optimization

This notebook analyzes the relationship between effective capacitance, operating frequency, and induced gas velocity in Dielectric Barrier Discharge (DBD) plasma actuators.

Based on the empirical equation:
$$v = \frac{(8.64 \times 10^{-4})^2}{\rho} \cdot \frac{1}{\omega C_{eff}}$$

Where:
- $v$ = induced gas velocity (m/s)
- $\rho$ = fluid density (kg/m³)
- $\omega$ = angular frequency (rad/s)
- $C_{eff}$ = effective capacitance (F)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from matplotlib.ticker import ScalarFormatter

# Core equation from your paper (Eq. 1)
def induced_velocity(C_eff, f, rho=1.225):
    """
    Calculate induced gas velocity from DBD actuator parameters
    Args:
        C_eff : Effective capacitance (F)
        f     : Operating frequency (Hz)
        rho   : Fluid density (kg/m³, default=air at sea level)
    Returns:
        v : Induced velocity (m/s)
    """
    omega = 2 * np.pi * f  # Angular frequency
    const = (8.64e-4)**2  # Empirical constant from your paper
    return (const / rho) * (1 / (omega * C_eff))

# Paper validation case (from Section III)
paper_C_eff = 62e-12  # 62 pF
paper_freq = 1000      # 1 kHz (typical DBD frequency)
paper_velocity = induced_velocity(paper_C_eff, paper_freq)

print(f"Paper validation case (C_eff = 62 pF, f = 1 kHz):")
print(f"Calculated velocity = {paper_velocity:.2f} m/s")

# Create interactive exploration
plt.figure(figsize=(12, 8))

def plot_velocity_relationships(C_base=62e-12, f_base=1000):
    # Capacitance sweep (50% reduction demonstration)
    C_range = np.linspace(C_base, 0.5*C_base, 50)
    v_C = induced_velocity(C_range, f_base)

    # Frequency sweep
    f_range = np.linspace(500, 5000, 50)
    v_f = induced_velocity(C_base, f_range)

    # Create plots
    plt.subplot(2, 2, 1)
    plt.plot(C_range*1e12, v_C, 'b-', linewidth=2)
    plt.axvline(x=C_base*1e12, color='r', linestyle='--', alpha=0.5)
    plt.axvline(x=0.5*C_base*1e12, color='g', linestyle='--', alpha=0.5)
    plt.xlabel('Capacitance $C_{eff}$ (pF)')
    plt.ylabel('Induced Velocity $v$ (m/s)')
    plt.title('Velocity vs Capacitance (50% Reduction Effect)')
    plt.grid(True, which='both', linestyle='--', alpha=0.7)
    plt.annotate(f'{v_C[0]:.1f} m/s → {v_C[-1]:.1f} m/s\n({v_C[-1]/v_C[0]:.1f}x increase)',
                xy=(0.5*C_base*1e12, v_C[-1]),
                xytext=(0.5*C_base*1e12-10, v_C[-1]+0.5),
                arrowprops=dict(facecolor='black', shrink=0.05))

    plt.subplot(2, 2, 2)
    plt.plot(f_range, v_f, 'r-', linewidth=2)
    plt.axvline(x=f_base, color='b', linestyle='--', alpha=0.5)
    plt.xlabel('Frequency $f$ (Hz)')
    plt.ylabel('Induced Velocity $v$ (m/s)')
    plt.title('Velocity vs Frequency')
    plt.grid(True, which='both', linestyle='--', alpha=0.7)

    # 3D surface plot (C_eff vs f)
    plt.subplot(2, 2, (3,4))
    C_vals = np.logspace(np.log10(10e-12), np.log10(200e-12), 50)
    f_vals = np.linspace(500, 5000, 50)
    C_grid, f_grid = np.meshgrid(C_vals, f_vals)
    v_grid = induced_velocity(C_grid, f_grid)

    cont = plt.contourf(C_grid*1e12, f_grid, v_grid, 20, cmap='viridis')
    plt.colorbar(cont, label='Velocity (m/s)')
    plt.plot(paper_C_eff*1e12, paper_freq, 'ro', markersize=8, label='Paper Configuration')
    plt.xlabel('Capacitance $C_{eff}$ (pF)')
    plt.ylabel('Frequency $f$ (Hz)')
    plt.title('Velocity Contours: $C_{eff}$ vs $f$')
    plt.legend()
    plt.yscale('log')
    plt.gca().xaxis.set_major_formatter(ScalarFormatter())

    plt.tight_layout()
    plt.show()

# Create interactive sliders
interact(
    plot_velocity_relationships,
    C_base=FloatSlider(min=10e-12, max=200e-12, step=10e-12, value=62e-12,
                       description='Base $C_{eff}$ (F)', readout_format='.1e'),
    f_base=FloatSlider(min=500, max=5000, step=100, value=1000,
                       description='Base $f$ (Hz)')
)

Paper validation case (C_eff = 62 pF, f = 1 kHz):
Calculated velocity = 1.56 m/s


<Figure size 1200x800 with 0 Axes>

interactive(children=(FloatSlider(value=6.2e-11, description='Base $C_{eff}$ (F)', max=2e-10, min=1e-11, reado…

<function __main__.plot_velocity_relationships(C_base=6.2e-11, f_base=1000)>

## Key Insights

1. **Inverse Relationship**: Velocity is inversely proportional to both capacitance and frequency
2. **Capacitance Optimization**: Reducing effective capacitance by 50% doubles the induced velocity
3. **Frequency Trade-offs**: Lower frequencies increase velocity but may affect plasma stability
4. **Design Space**: The contour plot shows the full parameter space for optimization

## Applications

- Flow control optimization
- Actuator design parameter selection
- Performance prediction
- Energy efficiency analysis